In [1]:


from torch.utils.data import Dataset
from PIL import Image
import os
import pandas as pd
import matplotlib.pylab as plt
import torchvision.transforms.functional as TF
import numpy as np
import random
from PIL import Image, ImageDraw
from torchvision import utils

path2data="/Users/mithindev/Desktop/COLLEGE/SEM-5/DL/CODE/data"
path2train="/Users/mithindev/Desktop/COLLEGE/SEM-5/DL/CODE/data/train"


#path2labels="/media/mithun/New Volume/2021_pc/disc/code/drion_training.xlsx"
# path2labels=os.path.join(path2data,"drion_training.xlsx")
# labels_df=pd.read_excel(path2labels)
# labels_df.head()
# labels_df.tail()
# print(labels_df['label'].value_counts())
# imgName=labels_df["imgName"]



# def load_img_label(labels_df,id_):   
    # imgName=labels_df["imgName"]
    # fullPath2img=os.path.join(path2data,"training_drion",imgName[id_])
    # img = Image.open(fullPath2img)
    # x=labels_df["label"][id_]
    # label=x 
    # return img,label 
    
import pandas as pd
from PIL import Image

def create_labels_dataframe(path2data, excel_path):
    data = []
    for category in ['cats', 'dogs']:
        folder_path = os.path.join(path2data, category)
        for img_name in os.listdir(folder_path):
            if img_name.endswith('.jpg'):
                img_path = os.path.join(folder_path, img_name)
                label = 0 if category == 'cats' else 1  # Assign label: 0 for cats, 1 for dogs
                data.append((img_path, label))
    
    labels_df = pd.DataFrame(data, columns=['img_path', 'label'])
    labels_df.to_excel(excel_path, index=False)  # Save to Excel file
    return labels_df

def my_load_img_labels(excel_path, id_):
    labels_df = pd.read_excel(excel_path)
    img_path = labels_df["img_path"].iloc[id_]
    img = Image.open(img_path)
    label = labels_df["label"].iloc[id_]
    return img, label

class AMD_dataset(Dataset):
      def __init__(self, path2data,transform):
           path2labels=os.path.join(path2data,"drion_training.xlsx")
           labels_df=pd.read_excel(path2labels)
           self.labels = labels_df[["label"]].values
           self.imgName=labels_df["imgName"]
           self.ids=labels_df.index
           self.fullPath2img=[0]*len(self.ids)
           for id_ in self.ids:
               self.fullPath2img[id_-1]=os.path.join(path2data,"training_drion",imgName[id_])
           self.transform = transform
                    
           
      def __len__(self):
          return len(self.labels)
      
      def __getitem__(self, idx):
          #image = Image.open(self.fullPath2img)
          image = Image.open(self.fullPath2img[idx],mode="r")
          label= self.labels[idx]
          #image = self.transform(image)
          image,label = self.transform(image,label)
          return image, self.labels[idx]
     
        
def transformer(image, label):
    image=TF.to_tensor(image)
    return image, label

amd_ds1=AMD_dataset(path2data,transformer)
amd_ds2=AMD_dataset(path2data,transformer)
print(len(amd_ds1))

from sklearn.model_selection import ShuffleSplit
sss = ShuffleSplit(n_splits=1, test_size=0.2, random_state=0)
indices=range(len(amd_ds1))
for train_index, val_index in sss.split(indices):
    print(len(train_index))
    print("-"*10)
    print(len(val_index))
from torch.utils.data import Subset
train_ds=Subset(amd_ds1,train_index)
print(len(train_ds))
val_ds=Subset(amd_ds2,val_index)
print(len(val_ds))


def show(img,label):
    npimg = np.array(img)
    npimg_tr=np.transpose(npimg, (1,2,0))
    plt.imshow(npimg_tr)
    #plt.imshow(np.asarray(img))
    if label is not None:
        x=label
        plt.plot(x,'b+',markersize=30)


plt.figure(figsize=(5,5))
for img,label in train_ds:
    show(img,label)
    break 

plt.figure(figsize=(5,5))
for img,label in val_ds:
    show(img,label)
    break  

import torch
from torch.utils.data import DataLoader
train_dl = DataLoader(train_ds, batch_size=5, shuffle=True)
val_dl = DataLoader(val_ds, batch_size=3, shuffle=True)

for img_b, label_b in train_dl:
    print(img_b.shape,img_b.dtype)
    print(label_b.shape)
    break

for img_b, label_b in val_dl:
    print(img_b.shape,img_b.dtype)
    #label_b=torch.stack(label_b,1)
    label_b=label_b.type(torch.float32)
    print(label_b.shape,label_b.dtype)
    break

for x,y in train_dl:
    print(x.shape,y)
    break
for x, y in train_dl:
    print(x.shape)
    print(y.shape)
    
def accuracy(labels, out):
    return np.sum(out==labels)/float(len(labels))

import torch.nn as nn          
def findConv2dOutShape(H_in,W_in,conv,pool=2):# get conv arguments
    kernel_size=conv.kernel_size
    stride=conv.stride
    padding=conv.padding
    dilation=conv.dilation
    H_out=np.floor((H_in+2*padding[0]-dilation[0]*(kernel_size[0]-1)-1)/stride[0]+1)
    W_out=np.floor((W_in+2*padding[1]-dilation[1]*(kernel_size[1]-1)-1)/stride[1]+1)
    if pool:
       H_out/=pool
       W_out/=pool
    return int(H_out),int(W_out) 


import torch.nn.functional as F
class Net(nn.Module):
      def __init__(self, params):
          super(Net, self).__init__()
          C_in,H_in,W_in=params["input_shape"]
          init_f=params["initial_filters"]
          num_fc1=params["num_fc1"]
          num_classes=params["num_classes"]
          self.dropout_rate=params["dropout_rate"]
          self.conv1 = nn.Conv2d(C_in, init_f, kernel_size=3)
          h,w=findConv2dOutShape(H_in,W_in,self.conv1)
          self.conv2 = nn.Conv2d(init_f, 2*init_f, kernel_size=3)
          h,w=findConv2dOutShape(h,w,self.conv2)
          self.conv3 = nn.Conv2d(2*init_f, 4*init_f, kernel_size=3)
          h,w=findConv2dOutShape(h,w,self.conv3)
          self.conv4 = nn.Conv2d(4*init_f, 8*init_f, kernel_size=3)
          h,w=findConv2dOutShape(h,w,self.conv4)
# compute the flatten size
          self.num_flatten=h*w*8*init_f
          self.fc1 = nn.Linear(self.num_flatten, num_fc1)
          self.fc2 = nn.Linear(num_fc1, num_classes)
      def forward(self, x):
          x = F.relu(self.conv1(x))
          x = F.max_pool2d(x, 2, 2)
          x = F.relu(self.conv2(x))
          x = F.max_pool2d(x, 2, 2)
          x = F.relu(self.conv3(x))
          x = F.max_pool2d(x, 2, 2)
          x = F.relu(self.conv4(x))
          x = F.max_pool2d(x, 2, 2)
          x = x.view(-1, self.num_flatten)
          x = F.relu(self.fc1(x))
          x=F.dropout(x, self.dropout_rate, training= self.training)
          x = self.fc2(x)
          return F.log_softmax(x,dim=1)
params_model={
"input_shape": (3,70,70),
"initial_filters": 8,
"num_fc1": 100,
"dropout_rate": 0.25,
"num_classes": 2,
}
model = Net(params_model)
if torch.cuda.is_available():
   device = torch.device("cuda")
   model=model.to(device)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# cnn_model = Net(params_model)
# cnn_model = cnn_model.to(device)
print(model)
print(next(model.parameters()).device)

from torchsummary import summary
summary(model, input_size=(3, 70, 70),device=device.type)

FileNotFoundError: [Errno 2] No such file or directory: '/Users/mithindev/Desktop/COLLEGE/SEM-5/DL/CODE/data/drion_training.xlsx'

In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Thu Feb 29 09:29:05 2024

@author: mithun
"""

from torch.utils.data import Dataset
from PIL import Image
import os
import pandas as pd
import matplotlib.pylab as plt
import torchvision.transforms.functional as TF
import numpy as np
import random
from PIL import Image, ImageDraw
from torchvision import utils

path2data="E:/Amrita_2024/DEEP LEARNING FOR SIGNAL & IMAGE PROCESSING/Python_neural network/"
path2train="E:/Amrita_2024/DEEP LEARNING FOR SIGNAL & IMAGE PROCESSING/Python_neural network/training_drion/"
#path2labels="/media/mithun/New Volume/2021_pc/disc/code/drion_training.xlsx"
path2labels=os.path.join(path2data,"drion_training.xlsx")
labels_df=pd.read_excel(path2labels)
labels_df.head()
labels_df.tail()
print(labels_df['label'].value_counts())
imgName=labels_df["imgName"]



# def load_img_label(labels_df,id_):   
#     imgName=labels_df["imgName"]
#     fullPath2img=os.path.join(path2data,"training_drion",imgName[id_])
#     img = Image.open(fullPath2img)
#     x=labels_df["label"][id_]
#     label=x 
#     return img,label 
# def show1(img,label):
#     #npimg = img.numpy().transpose((1,2,0))
#     npimg = img.numpy()
#     npimg_tr=np.transpose(npimg, (1,2,0))
#     plt.imshow(npimg_tr)


class AMD_dataset(Dataset):
      def __init__(self, path2data,transform):
           path2labels=os.path.join(path2data,"drion_training.xlsx")
           labels_df=pd.read_excel(path2labels)
           self.labels = labels_df[["label"]].values
           self.imgName=labels_df["imgName"]
           self.ids=labels_df.index
           self.fullPath2img=[0]*len(self.ids)
           for id_ in self.ids:
               self.fullPath2img[id_-1]=os.path.join(path2data,"training_drion",imgName[id_])
           self.transform = transform
                    
           
      def __len__(self):
          return len(self.labels)
      
      def __getitem__(self, idx):
          #image = Image.open(self.fullPath2img)
          image = Image.open(self.fullPath2img[idx],mode="r")
          label= self.labels[idx]
          #image = self.transform(image)
          image,label = self.transform(image,label)
          return image, self.labels[idx]
     
        
def transformer(image, label):
    image=TF.to_tensor(image)
    return image, label

amd_ds1=AMD_dataset(path2data,transformer)
amd_ds2=AMD_dataset(path2data,transformer)
print(len(amd_ds1))

from sklearn.model_selection import ShuffleSplit
sss = ShuffleSplit(n_splits=1, test_size=0.2, random_state=0)
indices=range(len(amd_ds1))
for train_index, val_index in sss.split(indices):
    print(len(train_index))
    print("-"*10)
    print(len(val_index))
from torch.utils.data import Subset
train_ds=Subset(amd_ds1,train_index)
print(len(train_ds))
val_ds=Subset(amd_ds2,val_index)
print(len(val_ds))


def show(img,label):
    npimg = np.array(img)
    npimg_tr=np.transpose(npimg, (1,2,0))
    plt.imshow(npimg_tr)
    #plt.imshow(np.asarray(img))
    if label is not None:
        x=label
        plt.plot(x,'b+',markersize=30)


plt.figure(figsize=(5,5))
for img,label in train_ds:
    show(img,label)
    break 

plt.figure(figsize=(5,5))
for img,label in val_ds:
    show(img,label)
    break  

import torch
from torch.utils.data import DataLoader
train_dl = DataLoader(train_ds, batch_size=5, shuffle=True)
val_dl = DataLoader(val_ds, batch_size=3, shuffle=True)

for img_b, label_b in train_dl:
    print(img_b.shape,img_b.dtype)
    print(label_b.shape)
    break

for img_b, label_b in val_dl:
    print(img_b.shape,img_b.dtype)
    #label_b=torch.stack(label_b,1)
    label_b=label_b.type(torch.float32)
    print(label_b.shape,label_b.dtype)
    break

for x,y in train_dl:
    print(x.shape,y)
    break
for x, y in train_dl:
    print(x.shape)
    print(y.shape)
    
def accuracy(labels, out):
    return np.sum(out==labels)/float(len(labels))

import torch.nn as nn          
def findConv2dOutShape(H_in,W_in,conv,pool=2):# get conv arguments
    kernel_size=conv.kernel_size
    stride=conv.stride
    padding=conv.padding
    dilation=conv.dilation
    H_out=np.floor((H_in+2*padding[0]-dilation[0]*(kernel_size[0]-1)-1)/stride[0]+1)
    W_out=np.floor((W_in+2*padding[1]-dilation[1]*(kernel_size[1]-1)-1)/stride[1]+1)
    if pool:
       H_out/=pool
       W_out/=pool
    return int(H_out),int(W_out) 


import torch.nn.functional as F
class Net(nn.Module):
      def __init__(self, params):
          super(Net, self).__init__()
          C_in,H_in,W_in=params["input_shape"]
          init_f=params["initial_filters"]
          num_fc1=params["num_fc1"]
          num_classes=params["num_classes"]
          self.dropout_rate=params["dropout_rate"]
          self.conv1 = nn.Conv2d(C_in, init_f, kernel_size=3)
          h,w=findConv2dOutShape(H_in,W_in,self.conv1)
          self.conv2 = nn.Conv2d(init_f, 2*init_f, kernel_size=3)
          h,w=findConv2dOutShape(h,w,self.conv2)
          self.conv3 = nn.Conv2d(2*init_f, 4*init_f, kernel_size=3)
          h,w=findConv2dOutShape(h,w,self.conv3)
          self.conv4 = nn.Conv2d(4*init_f, 8*init_f, kernel_size=3)
          h,w=findConv2dOutShape(h,w,self.conv4)
# compute the flatten size
          self.num_flatten=h*w*8*init_f
          self.fc1 = nn.Linear(self.num_flatten, num_fc1)
          self.fc2 = nn.Linear(num_fc1, num_classes)
      def forward(self, x):
          x = F.relu(self.conv1(x))
          x = F.max_pool2d(x, 2, 2)
          x = F.relu(self.conv2(x))
          x = F.max_pool2d(x, 2, 2)
          x = F.relu(self.conv3(x))
          x = F.max_pool2d(x, 2, 2)
          x = F.relu(self.conv4(x))
          x = F.max_pool2d(x, 2, 2)
          x = x.view(-1, self.num_flatten)
          x = F.relu(self.fc1(x))
          x=F.dropout(x, self.dropout_rate, training= self.training)
          x = self.fc2(x)
          return F.log_softmax(x,dim=1)
params_model={
"input_shape": (3,70,70),
"initial_filters": 8,
"num_fc1": 100,
"dropout_rate": 0.25,
"num_classes": 2,
}
model = Net(params_model)
if torch.cuda.is_available():
   device = torch.device("cuda")
   model=model.to(device)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# cnn_model = Net(params_model)
# cnn_model = cnn_model.to(device)
print(model)
print(next(model.parameters()).device)

from torchsummary import summary
summary(model, input_size=(3, 70, 70),device=device.type)